In [ ]:
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torch
import re
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AdamW
from transformers import AutoTokenizer, LongT5Model
import torch.nn.functional as F
import torch.nn as nn
import re
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
import torch
import torch.nn as nn

class RNNCell(torch.nn.Module):

    def __init__(self, input_size, hidden_size):
        super(RNNCell, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.input_weights = nn.Linear(self.input_size, self.hidden_size)
        self.hidden_weights = nn.Linear(self.hidden_size, self.hidden_size)

    def forward(self, sequence):
        batch_size, seq_length, _ = sequence.size()
        hidden_state = torch.zeros((batch_size, self.hidden_size), device=sequence.device)

        for i in range(seq_length):
            XI = self.input_weights(sequence[:, i, :])
            XH = self.hidden_weights(hidden_state)
            hidden_state = torch.tanh(XH + XI)

        return hidden_state

class MultiOutputRNNCell(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_size):
        super(MultiOutputRNNCell, self).__init__()

        self.input_size = embedding_dim
        self.hidden_size = hidden_size
        self.input_weights = nn.Linear(self.input_size, self.hidden_size)
        self.hidden_weights = nn.Linear(self.hidden_size, self.hidden_size)

    def forward(self, sequence):
        # batch_size, seq_length is (64, 38, embedding_dim)
        batch_size, seq_length, embedding_dim = sequence.size()
        # shape is (batch_size * hidden_size)
        hidden_state = torch.zeros((batch_size, self.hidden_size), device=sequence.device)
        return_sequences = []

        for i in range(seq_length):
            # word shape is (batch_size, embedding_dim)
            chars = sequence[:, i, embedding_dim]
            #nn.Linear can accept batch * dim
            XI = self.input_weights(chars)
            XH = self.hidden_weights(hidden_state)
            hidden_state = torch.tanh(XH + XI)
            return_sequences.append(hidden_state)
        return_sequences = torch.stack(return_sequences, dim=1)  # Shape: [batch_size, seq_length, hidden_size]
        return return_sequences

class CharLM(torch.nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size):
        super(CharLM, self).__init__()
        self.rnn_cell = MultiOutputRNNCell(input_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, vocab_size)
        self.vocab_size = vocab_size

    def forward(self, sequence):
        rnn_outputs = self.rnn_cell(sequence)
        logits = self.output_layer(rnn_outputs)
        return logits

class CharLMDataset(Dataset):
    def __init__(self, input_chars, output_chars, char_to_index):
        self.input_chars = input_chars
        self.output_chars = output_sequence
        self.char_to_index = char_to_index

    def __len__(self):
        return len(self.input_chars)

    def __getitem__(self, idx):
        input_char, target_char = self.input_chars[idx], self.output_chars[idx]
        input_char_idx = self.char_to_index[input_char]
        target_char_idx = self.char_to_index[target_char]
        input_char_tensor = self.one_hot_encode(input_char_idx, len(self.char_to_index))
        target_char_tensor = self.one_hot_encode(target_char_idx, len(self.char_to_index))
        return input_char_tensor, target_char_tensor

    def one_hot_encode(self, index, vocab_size):
      one_hot_vector = np.zeros(vocab_size)
      one_hot_vector[index] = 1
      return torch.tensor(one_hot_vector, dtype=torch.float)

In [ ]:
# create dataset
# input sequence is a
# output sequence is t
article_sentences = [
    "In today’s rapidly evolving world, the concept of lifelong learning has gained significant importance.",
    "The pace at which technology, knowledge, and societal norms are advancing necessitates continuous personal and professional development.",
    "Lifelong learning is no longer a choice but a necessity for individuals who seek to remain relevant and competitive in their fields.",
    "The traditional model of education, where learning is confined to the early stages of life, is becoming increasingly obsolete.",
    "In the past, a person could complete their formal education, enter the workforce, and rely on the skills and knowledge acquired during their schooling to sustain a lifelong career.",
    "However, this model is no longer viable in the 21st century.",
    "The rapid advancements in technology, particularly in fields such as artificial intelligence, data science, and biotechnology, mean that skills and knowledge can quickly become outdated.",
    "As a result, professionals must continually update their skill sets to keep pace with these changes.",
    "Lifelong learning is not just about acquiring new technical skills; it also involves personal growth and development.",
    "Engaging in continuous learning can enhance cognitive abilities, foster creativity, and improve problem-solving skills.",
    "These attributes are invaluable in both professional and personal contexts.",
    "By embracing lifelong learning, individuals can cultivate a mindset of curiosity and adaptability, which is essential for navigating the complexities of modern life.",
    "Moreover, lifelong learning plays a crucial role in career advancement.",
    "In an increasingly competitive job market, employers value individuals who demonstrate a commitment to continuous improvement.",
    "Professionals who engage in ongoing education and skill development are more likely to secure promotions, higher salaries, and leadership positions.",
    "Additionally, lifelong learning can open up new career opportunities, enabling individuals to pivot to different industries or roles as their interests and the job market evolve.",
    "The rise of online learning platforms has made lifelong learning more accessible than ever before.",
    "With the advent of massive open online courses (MOOCs), webinars, and online certifications, individuals can learn at their own pace and from the comfort of their homes.",
    "These platforms offer a wide range of courses covering various subjects, from coding and data analysis to leadership and creative writing.",
    "The flexibility and affordability of online learning have democratized education, allowing people from all walks of life to pursue their learning goals.",
    "Beyond professional development, lifelong learning has significant societal benefits.",
    "An educated and informed populace is better equipped to make decisions that positively impact society.",
    "Lifelong learners are more likely to engage in civic activities, volunteerism, and community service, contributing to the well-being of their communities.",
    "Additionally, lifelong learning promotes social inclusion by providing opportunities for individuals from diverse backgrounds to acquire knowledge and skills that can improve their quality of life.",
    "In conclusion, lifelong learning is essential in today’s fast-paced and ever-changing world.",
    "It empowers individuals to remain relevant in their careers, fosters personal growth, and contributes to the betterment of society.",
    "By embracing lifelong learning, we can cultivate a culture of curiosity, adaptability, and continuous improvement, ensuring that we are well-prepared to meet the challenges and opportunities of the future."
]

article_sentences= " ".join(article_sentences)
article_sentences = article_sentences.lower()

In [ ]:
char_list = list(set(article_sentences))
char_list.append('<start>')
char_list.append('<end>')
char_to_index = {char: index for index, char in enumerate(char_list)}
index_to_char = {index: char for char, index in char_to_index.items()}
vocab_size = len(char_to_index)

In [ ]:
from types import prepare_class
input_sequence = []
output_sequence = []

for i in range(len(article_sentences)-1):
    input_sequence.append(article_sentences[i])
    output_sequence.append(article_sentences[i+1])
    try:
      if article_sentences[i] == '.':
        input_sequence.append('.')
        output_sequence.append('<end>')
        if article_sentences[i+1] == ' ' :
          output_sequence.append(article_sentences[i+2])
          input_sequence.append('<start>')
    except:
      pass


In [ ]:
from torch.utils.data import DataLoader, Dataset
training_data = CharLMDataset(input_sequence, output_sequence, char_to_index)
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn

model = CharLM(vocab_size=vocab_size, input_size=vocab_size, hidden_size=100)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs = 5

for epoch in range(num_epochs):
    for input_char, target_char in train_dataloader:
        logits = model(input_char)
        logits_flat = logits.view(-1, vocab_size)
        targets_flat =  target_char.view(-1)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(logits_flat, targets_flat)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

64 38


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x64 and 38x100)